In [ ]:
import delta

def merge_data(input_df, container, database, table, file_format, partition_col = None, merge_condition = ''):
    tables = [db_table.name for db_table in spark.catalog.listTables(database)]
     
    if table in tables:
        delta_table = delta.DeltaTable.forPath(spark, f'synfs:/{job_id}/mnt/{container}/{table}')

        delta_table.alias('tgt') \
            .merge(
                input_df.alias('src'),
                merge_condition
            ) \
            .whenMatchedUpdateAll() \
            .whenNotMatchedInsertAll() \
            .execute()
    else:
        if partition_col is None:
            input_df.write \
                .mode('overwrite') \
                .format(file_format) \
                .saveAsTable(f'{database}.{table}')
        else:
            input_df.write \
                .mode('overwrite') \
                .format(file_format) \
                .partitionBy(partition_col) \
                .saveAsTable(f'{database}.{table}')

In [ ]:
def create_mounts():
    containers = ['bronze', 'silver', 'gold']
    mounts = mssparkutils.fs.mounts()

    
    for container in containers:
        mount_point = '/mnt/' + container
        mount_exists = any(mount.mountPoint == mount_point for mount in mounts)

        if not mount_exists:
            mssparkutils.fs.mount( 
            f'abfss://{container}@prizepicksnba.dfs.core.windows.net', 
            f'/mnt/{container}', 
            {'LinkedService': 'ls_adls_prizepicksnba'} 
) 